In [ ]:
import pandas as pd
import numpy as np

##Загрузка датасетов##

In [ ]:
spark_path = r'/content/drive/MyDrive/ЭкоТомск/practice3/spark.csv'
spark = pd.read_csv(spark_path, index_col=0)

In [ ]:
efrsb_path = r'/content/drive/MyDrive/ЭкоТомск/practice3/efrsb.csv'
efrsb = pd.read_csv(efrsb_path, index_col=0, encoding="utf-16")

##Объединение данных##

In [ ]:
def merge_data(spark, efrsb):

  efrsb["Дата банкротства"]= efrsb["Дата банкротства"].astype("datetime64") #Изменение типа данных Даты банкротства
  data = spark.merge(efrsb, on=["Регистрационный номер", "Код налогоплательщика"], how='left') # Объединение информации

  #data.to_csv("/content/drive/MyDrive/ЭкоТомск/duplicate_data/duplicate_data.csv") # Сохранение данных о компаниях с несколькими датами банкротства

  data.drop_duplicates(subset=["Наименование", "Регистрационный номер", "Год"], keep = 'last', inplace = True)

  return data

In [ ]:
data = merge_data(spark, efrsb)

In [ ]:
data.info(show_counts = True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2846300 entries, 0 to 3104783
Data columns (total 40 columns):
 #   Column                                             Non-Null Count    Dtype  
---  ------                                             --------------    -----  
 0   Наименование                                       2846300 non-null  object 
 1   Регистрационный номер                              2846300 non-null  float64
 2   Код налогоплательщика                              2846300 non-null  float64
 3   Вид деятельности/отрасль                           2846300 non-null  object 
 4   Код основного вида деятельности                    2846300 non-null  object 
 5   Организационно-правовая форма                      2846300 non-null  object 
 6   Форма собственности                                2846300 non-null  object 
 7   Размер компании                                    2846258 non-null  object 
 8   Важная информация                                  337634 non-

##Добавление в датасет целевого столбца##

In [ ]:
years = pd.DatetimeIndex(data["Дата банкротства"].to_numpy()).year
years = years - data["Год"]
years = pd.Series(data=years)
years.fillna(0, inplace = True)
years = years.astype("int64")

In [ ]:
a = years[~np.isnan(years)]
print(a)

0          0
1          0
2          0
3          0
4          0
          ..
3104779    0
3104780    0
3104781    0
3104782    0
3104783    0
Length: 2846300, dtype: int64


In [ ]:
data["Банкротство"] = years

In [ ]:
data = data.drop("Дата банкротства", axis=1)

In [ ]:
data.to_csv("/content/drive/MyDrive/ЭкоТомск/practice3/bankruptcy_data.csv")

##Просмотр информации о компаниях -банкротах##

In [ ]:
dates = data.loc[data["Дата банкротства"].notnull(), ["Наименование", "Регистрационный номер", "Код налогоплательщика", "Год", "Дата банкротства", "Банкротство"]]

In [ ]:
dates.tail(20)

,Наименование,Регистрационный номер,Код налогоплательщика,Год,Дата банкротства,Банкротство
3102902,"СО КВАДРАТ, ООО",1.067310e+12,7.310102e+09,2020,2022-03-31,2
3103157,"СТРОИТЕЛЬ, ООО",1.126679e+12,6.679017e+09,2020,2022-05-04,2
3103218,"СТРОЙМОНТАЖ, ООО",1.140280e+12,2.730958e+08,2020,2022-03-23,2
3103251,"СТРОЙЭКСПЕРТ, ООО",1.160280e+12,2.590135e+08,2020,2022-04-22,2
3103337,"ТАТНЕФТЬ-НК-ОЙЛ, ООО",1.021603e+12,1.651027e+09,2020,2022-05-05,2
3103459,"ТЕХНОСТРОЙ, ООО",1.040101e+12,1.050433e+08,2020,2022-05-13,2
3103619,"ТРАНСРЕСУРС, ООО",1.142901e+12,2.901247e+09,2020,2022-05-04,2
3103730,"УГЗ, ООО",1.057425e+12,7.453148e+09,2020,2022-05-25,2
3103787,"УМ-1, ООО",1.023501e+12,3.525100e+09,2020,2022-05-19,2
3103817,"УПРАВЛЯЮЩАЯ КОМПАНИЯ №3, ООО",1.084313e+12,4.313007e+09,2020,2022-04-26,2
